In [1]:
# default_exp riann

# riann

> API details.

In [1]:
#hide
from nbdev.showdoc import *

In [2]:
#export 
import onnxruntime as rt
import numpy as np
import pkg_resources

In [3]:
#export
class RIANN:
    """
    Real-time progress plots for Jupyter notebooks.
    Parameters
    ----------
    onnx_path : str, optional, default: ``None``
        Path to the RIANN onnx file. Defaults to the file provided with the package.
    """

    def __init__(self, onnx_path=None):
        if onnx_path is None:
            try:
                onnx_path = pkg_resources.resource_string(__name__, "riann.onnx")
            except: #pkg_resources fails if the code is executed in the jupyter-notebook
                onnx_path = 'riann/riann.onnx'
        
        self.session = rt.InferenceSession(onnx_path)
        self.h0 = np.zeros((2,1,1,200),dtype=np.float32)
        
    def predict(self,acc,gyr,fs):
        """
        Update plot with external x-values.
        Parameters
        ----------
        acc: numpy-array [sequence_length x 3]
            Acceleration data of the IMU. The axis order is x,y,z.
        gyr: numpy-array [sequence_length x 3]
            Gyroscope data of the IMU. The axis order is x,y,z.
        fs: float
            Samplingrate of the provided IMU data
            
        Returns
        -------
        attitude unit-quaternions [sequence_length x 4]
        """
        #prepare minibatch for runtime execution
        np_inp = np.concatenate([acc,gyr,np.tile(1/fs,(acc.shape[0],1))],axis=-1).astype(np.float32)[None,...]
        
        return self.session.run([], {'input': np_inp ,'h0':self.h0})[0][0]

In [4]:
sequence_length=100
acc = np.ones((sequence_length,3))
gyr = np.ones_like(acc)
fs = 200

In [5]:
riann = RIANN()
riann.predict(acc,gyr,fs).shape

(100, 4)

In [7]:
#hide
from nbdev.export import *
notebook2script()

Converted index.ipynb.
Converted riann.ipynb.
